# Attachment Protocol Example - Alice

## Role: 

This notebook tests the capablity to extend the basic controller to control a custom protocol developed in an aries agent. This protocol allows you to attach a file to a message.
Run this along side [Attachment Protocol Bob](http://localhost:8889/lab/tree/attachment.ipynb)

In [ ]:
%autoawait
import time
import asyncio

## Intialise a controller 

In [ ]:
from attachment_controller.attachment_controller import AttachmentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"
API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
attach_controller = AttachmentController(admin_url=ADMIN_URL, api_key=API_KEY, webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT, webhook_base=WEBHOOK_BASE)

## Confirm that an active connection exists

This can fail sometimes, there are fallback steps described in Bob's notebook.

In [ ]:
response = await attach_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':   
        global connection_id
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
    else:
        print("Connection is still progressing to active state, retry in a few moments")
else:
    print("You must create a connection")
    

### Note: We do not need to initialise any listeners for Alice. In this example she only sends attachments and does not receive them

## Make sure you have initialised the handler on [Bob's notebook](http://localhost:8889/lab/tree/attachment.ipynb)

## Send an attachment over DIDcomm to Bob

In this instance a text file is sent, this will be received and saved by the handler in the [Bob's notebook](http://localhost:8889/lab/tree/attachment.ipynb).

In [ ]:
from aiohttp import FormData
data = FormData()
data.add_field('file',
               open('test_file.txt', 'rb'),
               filename='test_file.txt',
               content_type='text/plain')
response = await attach_controller.protocol.send_attachment(connection_id, data)
print('Attachment- Alice-> Bob')
print(response)


Here an image file is sent and is received by the handler in [Bob's notebook](http://localhost:8889/lab/tree/attachment.ipynb).

In [ ]:
data = FormData()
data.add_field('file',
               open('openmined.jpg', 'rb'),
               filename='openmined.jpg',
               content_type='image/jpeg')
response = await attach_controller.protocol.send_attachment(connection_id, data)
print('Attachment- Alice-> Bob')
print(response)


# End of Tutorial

Be sure to terminate the controller so you can run another tutorial

In [ ]:
response = await attach_controller.terminate()
print(response)
